<a href="https://colab.research.google.com/github/kshwan95/Machine-Learning/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5_%EB%94%A5%EB%9F%AC%EB%8B%9D4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Embedding:
  def __init__(self,W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None
  def forward(self,idx):
    W, = self.params
    self.idx = idx
    out = W[idx]
    return out

In [2]:
def backward(self, dout):
  dW, = self.grads
  dW[...] = 0
  for i, word_id in enumerate(self.idx):
    dW[word_id]+=dout[i]
    return None

In [3]:
class EmbeddingDot:
  def __init__(self,W):
    self.embed = Embedding(W)
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None
  def forward(self, h ,idx):
    target_W = self.embed.forward(idx)
    out = np.sum(target_W*h,axis=1)
    self.cache = (h, target_W)
    return out
  def backward(self,dout):
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0],1)
    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

In [5]:
#네거티브 샘플링 구현
class NegativeSampligLoss:
  def __init__(self, W, corpus, power=0.75, sample_size=5):
    self.sample_size = sample_size
    self.sampler = UnigramSampler(corpus, power, sample_size)
    self.loss_layers = [SigmodidWithLoss() for _ in range (sample_size +1)]
    self.embed_dot_layers = [EmbeddingDot(W) for _ in range(smaple_size+1)]
    self.params, self.grads = [], []
    for layer in self.embed_dot_layers:
      self.params += layer.params
      self.grads += layer.grads

In [7]:
def forward(self, h , target):
  batch_size = target.shape[0]
  negative_sample = self.sampler.get_negative_sample(target)

  #긍정적인 예 순전파
  score = self.embed_dot_layers[0].forward(h,target)
  correct_label = np.ones(batch_size, dtype=np.int32)
  loss = sefl.loss_layers[0].forward(score, correct_label)

  #부정적 예 순전파
  negative_label = np.zeros(batch_size, dtype=np.int32)
  for i in range(self.sample_size):
    negative_target = negative_sample[:,i]
    score = self.embed_dot_layers[1+i].forward(h,negative_target)
    loss += self.loss_layers[1 + i].forward(score, negative_label)
  return loss

In [11]:
def backward(self, dout=1):
  dh = 0
  for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
    dscore = l0.backward(dout)
    dh+= l1.backward(dscore)

    return dh

In [12]:
class CBOW:
  def __init__(self, vocab_size, hidden_size, window_size, corpus):
    V,H = vocab_size, hidden_size

    #가중치 초기화
    W_in = 0.01 * np.random.randn(V,H).astype('f')
    W_out = 0.01 * np.random.randn(V,H).astype('f')

    #계층 생성
    self.in_layers = []
    for i in range(2 * window_size):
      layer = Embedding(W_in)
      self.in_layers.append(layer)
    self.ns_loss = NegativeSampligLoss(W_out,corpus, power=0.75,sample_size=5)

    #모든 가중치와 기울기르 배열에 모은다
    layers = self.in_layers + [self.ns_loss]
    self.params, self.grads = [], []
    for layer in layers:
      self.params += layer.params
      self.grads += layer.grads
    
    #인스턴스 변수에 단어의 분산 표현을 저장한다.
    self.word_vecs = W_in